In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
from os.path import join as pjoin
from tqdm import tqdm
import plotly.graph_objects as go
import pingouin as pg

sys.path.append("../..")
import circletrack_behavior as ctb
import plotting_functions as pf

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [5]:
## Set parameters
behavior_path = '../../../MultiCon_AfterHours/'
fig_path = '../../../MultiCon_AfterHours/MultiCon_EEG1/intermediate_figures/'
num_experiments = 2 + 1 ## change first value since np.arange is not inclusive
experiment_list = ['MultiCon_EEG{}'.format(x) for x in np.arange(1, num_experiments)]
session_dict = {'mc_EEG1_01': ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5',
                               'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5'],
                'mc_EEG1_02': ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5',
                               'C1', 'C2', 'C3', 'C4', 'C5', 'A2_1', 'A2_2', 'A2_3', 'A2_4', 'A2_5'],
                'mc_EEG2_02': ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5',
                               'C1', 'C2', 'C3', 'C4', 'C5', 'A2_1', 'A2_2', 'A2_3', 'A2_4', 'A2_5']}

In [22]:
result_dict = {'mouse': [], 'day': [], 'accuracy': [], 'metric': []}
transitions = [4, 9, 14]
for experiment in experiment_list:
    bpath = pjoin(behavior_path, f'{experiment}/output/behav')
    for mouse in os.listdir(bpath):
        path = pjoin(bpath, mouse)
        for day in transitions:
            sess_one = pd.read_feather(pjoin(path, '{}_{}.feat'.format(mouse, session_dict[mouse][day])))
            sess_two = pd.read_feather(pjoin(path, '{}_{}.feat'.format(mouse, session_dict[mouse][day+1])))
            ## Rotate ports
            rot_one, rot_two = ctb.rotate_ports(input_maze=np.unique(sess_one['maze'])[0], 
                                                output_maze=np.unique(sess_two['maze'])[0], 
                                                reward_one=np.unique(sess_one['reward_one'])[0], 
                                                reward_two=np.unique(sess_one['reward_two'])[0])
            ## Calculate normalized probe metric with rotated ports
            metric = ctb.normalized_probe_metric(sess_two['lick_port'][(sess_two['probe']) & (sess_two['lick_port'] != -1)], 
                                                reward_one=rot_one,
                                                reward_two=rot_two)
            lick_acc = ctb.lick_accuracy(sess_two[sess_two['probe']], port_one=rot_one, port_two=rot_two)
            ## Add results
            result_dict['mouse'].append(mouse)
            result_dict['day'].append(day+2)
            result_dict['accuracy'].append(lick_acc)
            result_dict['metric'].append(metric)
probe_metric_df = pd.DataFrame(result_dict)
probe_metric_df['day'] = probe_metric_df['day'].replace({6: 'A to B', 11: 'B to C', 16: 'C to D'})

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../circletrack_behavior.py:1032: RuntimeWarning: invalid value encountered in scalar divide
  percent_correct = ((count_licks['first_licks'][(count_licks['lick_port'] == port_one) | (count_licks['lick_port'] == port_two)].dropna().sum()) /
/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/numpy/core/_metho

In [32]:
mouse_colors = ['turquoise', 'darkorchid', 'lightgreen']
fig = pf.custom_graph_template(x_title='', y_title='Lick Accuracy (%)')
for idx, mouse in enumerate(np.unique(probe_metric_df['mouse'])):
    sub_data = probe_metric_df[probe_metric_df['mouse'] == mouse]
    fig.add_trace(go.Scatter(x=sub_data['day'],
                            y=sub_data['accuracy'],
                            mode='markers',
                            marker = dict(color=mouse_colors[idx], line=dict(width = 1)),
                            name=mouse))
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color='darkgrey', opacity=1)
fig.show()
fig.write_image(pjoin(fig_path, 'rotated_probe_accuracy.png'))